In [1]:
import pandas as pd
import pyodbc
import numpy as np

In [2]:
import sqlCredentials as sql

In [3]:
sandbox = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=PROCUREMENTDB;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

In [4]:
pub_dict = """
SELECT *
FROM dbo.PublisherDictionary
"""

In [5]:
df = pd.read_sql(pub_dict,sandbox)

In [6]:
df.head()

,PublisherLong,PublisherShort
0,"""Abrams, Inc.""",ABRAMS
1,"""Addison-Wesley Longman, Incorporated""",PEARSON
2,"""Addison-Wesley Longman, Limited""",PEARSON
3,"""Allyn & Bacon, Incorporated""",PEARSON
4,"""American Occupational Therapy Association, In...","American Occupational Therapy Association, Inc..."


In [7]:
df.dtypes

PublisherLong     object
PublisherShort    object
dtype: object

In [8]:
megalist = """
SELECT *
FROM Process.SupplierMegaList
"""

In [9]:
megalist = pd.read_sql(megalist,sandbox)

In [10]:
megalist.head()

,ISBN,Supplier,ListName,Publisher,Currency,ListPrice,Discount,CostUnitPrice,OrderPercentFee,FlatRateFee,...,MaxCostPerOrder,MinCostPerOrder,ShipmentOrigin,Recurrency,Stockqty,UnitShippingCost,ExpirationDate,TotalUnitCost,Comment,Timestamp
0,9780262530873,BILLSON,pubs_matrix,UNIVERSITY PRESS,GBP,4.99,0.38,3.09,0.03,0.0,...,0.0,0.0,UK,weekly,3,3.47,2020-03-06,7.06,None,2020-03-24 18:00:00.817
1,9780262530989,BILLSON,pubs_matrix,UNIVERSITY PRESS,GBP,25.00,0.38,15.50,0.03,0.0,...,0.0,0.0,UK,weekly,3,3.47,2020-03-06,21.49,None,2020-03-24 18:00:00.817
2,9781760110185,ZOOKAL,Zookal Supplylist 3-23-2020,ALIEN & UNWIN,AUD,0.00,0.00,13.00,0.00,0.0,...,0.0,0.0,AUS,1 time,150,3.30,2020-09-19,10.82,None,2020-03-24 18:00:00.817
3,9780262531092,BILLSON,pubs_matrix,UNIVERSITY PRESS,GBP,37.95,0.38,23.53,0.03,0.0,...,0.0,0.0,UK,weekly,3,3.47,2020-03-06,30.82,None,2020-03-24 18:00:00.817
4,9780262531146,Bertrams,BertramsFTP3110,N/A,GBP,45.00,0.00,0.00,0.00,0.0,...,0.0,0.0,UK,weekly,0,3.47,2020-01-13,55.77,None,2020-03-24 18:00:00.817


In [11]:
megalist.dtypes

ISBN                        object
Supplier                    object
ListName                    object
Publisher                   object
Currency                    object
ListPrice                  float64
Discount                   float64
CostUnitPrice              float64
OrderPercentFee            float64
FlatRateFee                float64
LeadTimeWeeks               object
MaxQtyPerOrder               int64
MinQtyPerOrder               int64
MaxCostPerOrder            float64
MinCostPerOrder            float64
ShipmentOrigin              object
Recurrency                  object
Stockqty                     int64
UnitShippingCost           float64
ExpirationDate      datetime64[ns]
TotalUnitCost              float64
Comment                     object
Timestamp           datetime64[ns]
dtype: object

In [12]:
megalist.columns

Index(['ISBN', 'Supplier', 'ListName', 'Publisher', 'Currency', 'ListPrice',
       'Discount', 'CostUnitPrice', 'OrderPercentFee', 'FlatRateFee',
       'LeadTimeWeeks', 'MaxQtyPerOrder', 'MinQtyPerOrder', 'MaxCostPerOrder',
       'MinCostPerOrder', 'ShipmentOrigin', 'Recurrency', 'Stockqty',
       'UnitShippingCost', 'ExpirationDate', 'TotalUnitCost', 'Comment',
       'Timestamp'],
      dtype='object')

In [13]:
sup_list = megalist[['ISBN', 'Supplier', 'Publisher', 'Currency','ListPrice','Discount','CostUnitPrice', 'UnitShippingCost', 'MaxQtyPerOrder']]

In [14]:
sup_list.head()

,ISBN,Supplier,Publisher,Currency,ListPrice,Discount,CostUnitPrice,UnitShippingCost,MaxQtyPerOrder
0,9780262530873,BILLSON,UNIVERSITY PRESS,GBP,4.99,0.38,3.09,3.47,1000
1,9780262530989,BILLSON,UNIVERSITY PRESS,GBP,25.00,0.38,15.50,3.47,1000
2,9781760110185,ZOOKAL,ALIEN & UNWIN,AUD,0.00,0.00,13.00,3.30,150
3,9780262531092,BILLSON,UNIVERSITY PRESS,GBP,37.95,0.38,23.53,3.47,1000
4,9780262531146,Bertrams,N/A,GBP,45.00,0.00,0.00,3.47,100


In [15]:
sup_list['price'] = np.where(sup_list['ListPrice'] == 0.00, sup_list['CostUnitPrice'], sup_list['ListPrice'])

C:\Users\Jose\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
sup_list.head()

,ISBN,Supplier,Publisher,Currency,ListPrice,Discount,CostUnitPrice,UnitShippingCost,MaxQtyPerOrder,price
0,9780262530873,BILLSON,UNIVERSITY PRESS,GBP,4.99,0.38,3.09,3.47,1000,4.99
1,9780262530989,BILLSON,UNIVERSITY PRESS,GBP,25.00,0.38,15.50,3.47,1000,25.00
2,9781760110185,ZOOKAL,ALIEN & UNWIN,AUD,0.00,0.00,13.00,3.30,150,13.00
3,9780262531092,BILLSON,UNIVERSITY PRESS,GBP,37.95,0.38,23.53,3.47,1000,37.95
4,9780262531146,Bertrams,N/A,GBP,45.00,0.00,0.00,3.47,100,45.00


In [17]:
sup_list.drop(columns = ['ListPrice', 'CostUnitPrice'], inplace = True)

C:\Users\Jose\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [18]:
sup_list

,ISBN,Supplier,Publisher,Currency,Discount,UnitShippingCost,MaxQtyPerOrder,price
0,9780262530873,BILLSON,UNIVERSITY PRESS,GBP,0.38,3.47,1000,4.99
1,9780262530989,BILLSON,UNIVERSITY PRESS,GBP,0.38,3.47,1000,25.00
2,9781760110185,ZOOKAL,ALIEN & UNWIN,AUD,0.00,3.30,150,13.00
3,9780262531092,BILLSON,UNIVERSITY PRESS,GBP,0.38,3.47,1000,37.95
4,9780262531146,Bertrams,N/A,GBP,0.00,3.47,100,45.00
...,...,...,...,...,...,...,...,...
3320779,9780141191669,ALEK,RANDOM HOUSE,USD,0.64,0.50,5000,20.00
3320780,9780141191744,ALEK,RANDOM HOUSE,USD,0.64,0.50,5000,19.00
3320781,9780141191751,ALEK,RANDOM HOUSE,USD,0.64,0.50,5000,11.00
3320782,9780141191768,ALEK,RANDOM HOUSE,USD,0.64,0.50,5000,15.00


In [19]:
sup_list['Supplier'].unique()

array(['BILLSON', 'ZOOKAL', 'Bertrams', 'ALEK', 'ALEK ', 'Probook',
       'Billson', 'UT', 'Laurentiu', 'ABE', 'SENAD', 'Alek', 'Friday',
       'BRYAN', 'RAJAN', 'LAURENTIU', 'SOHRAB', 'Apex', 'laurentiu',
       'APEX', 'PROBOOK', 'Sohrab', 'Billson T', 'MYAMMAR', 'Bryan',
       'Textbook Trade HK', 'GAUTAM', 'WESTERN CAMPUS', 'Felipe/Armando',
       'FELIPE/ARMANDO', 'Armando', 'Billson - SCI'], dtype=object)

In [20]:
def clean_up(phrase):
    phrase = phrase.lstrip()
    phrase = phrase.rstrip()
    phrase = phrase.upper()
    return phrase

In [21]:
str_columns = ['ISBN', 'Supplier', 'Publisher', 'Currency']

for col in str_columns:
    sup_list[col] = sup_list.apply(lambda x: clean_up(x[col]), axis =1)
    
sup_list['Supplier'].unique()

C:\Users\Jose\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


array(['BILLSON', 'ZOOKAL', 'BERTRAMS', 'ALEK', 'PROBOOK', 'UT',
       'LAURENTIU', 'ABE', 'SENAD', 'FRIDAY', 'BRYAN', 'RAJAN', 'SOHRAB',
       'APEX', 'BILLSON T', 'MYAMMAR', 'TEXTBOOK TRADE HK', 'GAUTAM',
       'WESTERN CAMPUS', 'FELIPE/ARMANDO', 'ARMANDO', 'BILLSON - SCI'],
      dtype=object)

In [22]:
for col in str_columns:
    print(sup_list[col].unique())

['9780262530873' '9780262530989' '9781760110185' ... '9780141191577'
 '9780141191652' '9781760110178']
['BILLSON' 'ZOOKAL' 'BERTRAMS' 'ALEK' 'PROBOOK' 'UT' 'LAURENTIU' 'ABE'
 'SENAD' 'FRIDAY' 'BRYAN' 'RAJAN' 'SOHRAB' 'APEX' 'BILLSON T' 'MYAMMAR'
 'TEXTBOOK TRADE HK' 'GAUTAM' 'WESTERN CAMPUS' 'FELIPE/ARMANDO' 'ARMANDO'
 'BILLSON - SCI']
['UNIVERSITY PRESS' 'ALIEN & UNWIN' 'N/A' 'RANDOM HOUSE' 'FOOTPRINT'
 'ROWMAN & LITTLEFIELD' 'OXFORD' 'HARPER COLLINS'
 'PENGUIN RANDOM HOUSE USA' 'T&F' 'TF' 'EYE ON EDUCATION' 'PERIBO'
 'PRINCETON' 'SAGE' '"SAGE PUBLICATIONS, INC"' 'ALEK' 'CAMBRIDGE'
 'CUP ELT' 'CAMBRIDGE SEC. EDUCATION OTO' 'ADAMS MEDIA CORPORATION'
 'BEYOND WORDS PUBLISHING' 'EUROSPAN' 'DOVER' 'WILEY' 'CENGAGE' 'HEINLE'
 'FOCAL PR' 'FA DAVIS' 'ELSEVIER' 'ELSEVIER HEALTH'
 'GAZELLE BOOK SERVICES' 'MELIA PUBLISHING SERVICES LTD' 'CHELSEA G'
 'CHELSEA GREEN PUBLISHING CO' 'THIEME' 'SPRINGER' 'FINDHORN PRESS LTD'
 'MCGRAW' '"""ARCHIVE EDITIONS, LIMITED"""' 'LYNNE RIENNER' 'CICERONE'
 'UNI

In [24]:
# We have too many Publihser's name, we have to merge with the dictionary,
#Left Join would do the work
df.head()

,PublisherLong,PublisherShort
0,"""Abrams, Inc.""",ABRAMS
1,"""Addison-Wesley Longman, Incorporated""",PEARSON
2,"""Addison-Wesley Longman, Limited""",PEARSON
3,"""Allyn & Bacon, Incorporated""",PEARSON
4,"""American Occupational Therapy Association, In...","American Occupational Therapy Association, Inc..."


In [25]:
for col in list(df.columns):
    df[col] = df.apply(lambda x: clean_up(x[col]), axis =1)

In [27]:
df['PublisherShort'].unique()

array(['ABRAMS', 'PEARSON',
       'AMERICAN OCCUPATIONAL THERAPY ASSOCIATION, INCORPORATED',
       'ARCHIVE EDITIONS, LIMITED', 'AVIAT INC', 'BERRETT-KOEHLER',
       'FINANCIAL TIMES BUSINESS ENTERPRISES, LIMITED', 'WILEY', 'MCGRAW',
       'PIONEER DRAMA', 'MURACH', "O'REILLY", 'PLURAL PUBLISHING INC',
       'SCITECH PUBLISHING, INCORPORATED', 'SITEPOINT PTY, LIMITED',
       'SLACK', 'TIMBER PRESS',
       'DIFUSION CENTRO DE PUBLICACION Y PUBLICACIONES DE IDIOMAS, S.L.',
       'GAZELLE BOOK SERVICES', 'GROVE/ATLANTIC, INCORPORATED',
       'JAMES LORIMER & COMPANY LTD., PUBLISHERS', 'HACHETTE LIVE',
       'MASCOT BOOKS, INCORPORATED', 'TALONBOOKS, LIMITED',
       'A&C BLACK POD', 'AA WORLD', 'ABC CLIO', 'ABINGDON',
       'ADAMS MEDIA CORPORATION', 'AFRICAN & CARIBBEAN WRITERS', 'ALA',
       'ALEK', 'ALFRED MUSIC', 'ALLWORTH', 'AMA', 'AMER PSYCH ASS.',
       'AMAZON.COM', 'AMERICAN MATHEMATICAL SOCIETY', 'ANDREWS MCMEEL HB',
       'ANDREWS MCMEEL PB', 'ANGLO AMERICAN BOOK 

In [28]:
sup_list2 = pd.merge(sup_list, df, how='left', left_on = 'Publisher', right_on = 'PublisherLong' )

In [29]:
sup_list2

,ISBN,Supplier,Publisher,Currency,Discount,UnitShippingCost,MaxQtyPerOrder,price,PublisherLong,PublisherShort
0,9780262530873,BILLSON,UNIVERSITY PRESS,GBP,0.38,3.47,1000,4.99,UNIVERSITY PRESS,UNIVERSITY PRESS
1,9780262530989,BILLSON,UNIVERSITY PRESS,GBP,0.38,3.47,1000,25.00,UNIVERSITY PRESS,UNIVERSITY PRESS
2,9781760110185,ZOOKAL,ALIEN & UNWIN,AUD,0.00,3.30,150,13.00,ALIEN & UNWIN,ALIEN & UNWIN
3,9780262531092,BILLSON,UNIVERSITY PRESS,GBP,0.38,3.47,1000,37.95,UNIVERSITY PRESS,UNIVERSITY PRESS
4,9780262531146,BERTRAMS,N/A,GBP,0.00,3.47,100,45.00,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...
3320779,9780141191669,ALEK,RANDOM HOUSE,USD,0.64,0.50,5000,20.00,RANDOM HOUSE,RANDOM HOUSE
3320780,9780141191744,ALEK,RANDOM HOUSE,USD,0.64,0.50,5000,19.00,RANDOM HOUSE,RANDOM HOUSE
3320781,9780141191751,ALEK,RANDOM HOUSE,USD,0.64,0.50,5000,11.00,RANDOM HOUSE,RANDOM HOUSE
3320782,9780141191768,ALEK,RANDOM HOUSE,USD,0.64,0.50,5000,15.00,RANDOM HOUSE,RANDOM HOUSE


In [30]:
sup_list2.drop(columns = ['PublisherLong'], inplace = True)

In [35]:
print('# of pubs in PublishertShort Column: ', len(sup_list2['PublisherShort'].unique()))
print('# of pubs in Publisher Column: ', len(sup_list2['Publisher'].unique()))

# of pubs in PublishertShort Column:  319
# of pubs in Publisher Column:  747


In [32]:
sup_list2.head()

,ISBN,Supplier,Publisher,Currency,Discount,UnitShippingCost,MaxQtyPerOrder,price,PublisherShort
0,9780262530873,BILLSON,UNIVERSITY PRESS,GBP,0.38,3.47,1000,4.99,UNIVERSITY PRESS
1,9780262530989,BILLSON,UNIVERSITY PRESS,GBP,0.38,3.47,1000,25.00,UNIVERSITY PRESS
2,9781760110185,ZOOKAL,ALIEN & UNWIN,AUD,0.00,3.30,150,13.00,ALIEN & UNWIN
3,9780262531092,BILLSON,UNIVERSITY PRESS,GBP,0.38,3.47,1000,37.95,UNIVERSITY PRESS
4,9780262531146,BERTRAMS,N/A,GBP,0.00,3.47,100,45.00,N/A
